In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import datetime
from bs4 import BeautifulSoup
import csv

link = "https://search.pullpush.io/?kind=submission&subreddit=tifu&since=1688097600&until=1703998800&size=100000000"

search_keywords = link.split("&")
subreddit_name = search_keywords[1].split("=")[-1]

#Converting time stamp from Unix format to MM/DD/YY
start_dt = datetime.datetime.fromtimestamp(int(search_keywords[2].split("=")[-1]))
start =  start_dt.strftime('%m/%d/%y')

end_dt = datetime.datetime.fromtimestamp(int(search_keywords[3].split("=")[-1]))
end =  end_dt.strftime('%m/%d/%y')


# Function to check if the Fetch More button is present
def is_element_present(driver, xpath):
    try:
        element = driver.find_element(By.XPATH, xpath)
        return element.is_displayed()
    except NoSuchElementException:
        return False

# Initialize WebDriver
driver = webdriver.Chrome()

# driver.get("https://search.pullpush.io/?kind=submission&score=>1000&since=1604203200&until=1611118800&q=(America%20AND%20election*)%20OR%20(America*)%20OR%20%20OR%20(election*)%20%20OR%20((U*S*)%20AND%20election*)&size=10000")
driver.get(link)

# Wait for the search button to be clickable and click it
wait = WebDriverWait(driver, 60)
error_xpath = "//*[contains(text(), 'Search error. Pullpush may be down - ')]"
fetch_more_xpath = '//input[@value="Fetch More"]'

try:
    search_button = wait.until(EC.element_to_be_clickable((By.ID, 'searchButton')))
    search_button.click()
    while (is_element_present(driver, error_xpath)):
        search_button = wait.until(EC.element_to_be_clickable((By.ID, 'searchButton')))
        search_button.click()
    
except TimeoutException:
    print("Timeout waiting for search button to be clickable.")
    driver.quit()
    exit()

# Loop to click Fetch More and scroll down until it's no longer present
status = True
fetch_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Fetch More"]')))

while status:
    #fetch_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Fetch More"]')))
    if is_element_present(driver, fetch_more_xpath):
        try:
            fetch_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@value="Fetch More"]')))
            fetch_more_button.click()
            print("Clicked Fetch More button")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except TimeoutException:
            print("Timeout waiting for Fetch More button to be clickable.")
            break
    else:
        print("Fetch More button not found. Exiting loop.")
        status = False

# After loop, scroll to the bottom of the page one final time
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

results = driver.find_element(By.ID, 'results')
html_content = results.get_attribute('innerHTML')

soup = BeautifulSoup(html_content, 'html.parser')
cards = soup.find_all(class_='card')

# Prepare CSV file and write headers
csv_filename = f'{subreddit_name}-{start}-{end}-Extracted Links.csv'
csv_headers = ['Link']

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(csv_headers)
    # Extract href attributes from each card and write to CSV
    for card in cards:
        # Find all <a> tags within the card
        links = card.find_all('a', class_='is-pulled-right')
        for link in links:
            href = link.get('href')  # Get the value of the href attribute
            if href:
                writer.writerow([href])

print(f'Links extracted and saved to {csv_filename}')

# Close the WebDriver
driver.quit()


Clicked Fetch More button
Fetch More button not found. Exiting loop.


FileNotFoundError: [Errno 2] No such file or directory: 'tifu-06/30/23-12/31/23-Extracted Links.csv'